In [23]:
import pandas as pd
import numpy as np
import json
import re

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.neighbors import NearestNeighbors
from sklearn import neighbors
from sklearn.metrics import classification_report

from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error

In [203]:
with open('Data/reviews_Clothing_Shoes_and_Jewelry_5.json') as f:
    data = json.load(f)

In [238]:
df = pd.DataFrame.from_dict(data, orient='columns')

In [239]:
#df = df.sample(n=50000)
#df.to_csv('Data/reviews_50000.csv', encoding='utf-8', index=False)
#df = pd.read_csv('Data/reviews_50000.csv')
#helpful = df[df['helpful'] != '[0, 0]']
df

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0000031887,"[0, 0]",5.0,This is a great tutu and at a really great pri...,"02 12, 2011",A1KLRMWW2FWPL4,"Amazon Customer ""cameramom""",Great tutu- not cheaply made,1297468800
1,0000031887,"[0, 0]",5.0,I bought this for my 4 yr old daughter for dan...,"01 19, 2013",A2G5TCU2WDFZ65,Amazon Customer,Very Cute!!,1358553600
2,0000031887,"[0, 0]",5.0,What can I say... my daughters have it in oran...,"01 4, 2013",A1RLQXYNCMWRWN,Carola,I have buy more than one,1357257600
3,0000031887,"[0, 0]",5.0,"We bought several tutus at once, and they are ...","04 27, 2014",A8U3FAMSJVHS5,Caromcg,"Adorable, Sturdy",1398556800
4,0000031887,"[0, 0]",5.0,Thank you Halo Heaven great product for Little...,"03 15, 2014",A3GEOILWLK86XM,CJ,Grammy's Angels Love it,1394841600
5,0000031887,"[0, 0]",4.0,I received this today and I'm not a fan of it ...,"03 31, 2014",A27UF1MSF3DB2,"C-Lo ""Cynthia""",It's ok,1396224000
6,0000031887,"[0, 0]",5.0,Bought this as a backup to the regular ballet ...,"05 3, 2014",A16GFPNVF4Y816,design maven,Great for dress-up and for ballet practice,1399075200
7,0000031887,"[0, 0]",5.0,Great tutu for a great price. It isn't a &#34;...,"12 23, 2012",A2M2APVYIB2U6K,Jamie P.,Great value,1356220800
8,0000031887,"[0, 0]",4.0,"My daughter liked this, and it with her costum...","11 10, 2013",A1NJ71X3YPQNQ9,JBerger,Good,1384041600
9,0000031887,"[7, 8]",5.0,For what I paid for two tutus is unbeatable an...,"10 7, 2012",A3EERSWHAI6SO,"Jeffrey Hollingshead ""Jillian hollingshead""",WOW !! ..is all I have to say!,1349568000


In [208]:
count = df.groupby('asin', as_index=False).count()
mean = df.groupby('asin', as_index=False).mean()
df_merged = pd.merge(df, count, how='right', on=['asin'])

In [240]:
df_merged['totalReviewers'] = df_merged['reviewerID_y']
df_merged["overallScore"] = df_merged["overall_x"]
df_merged["summaryReview"] = df_merged["summary_x"]
# df_merged["textReview"] = df_merged[["reviewText_x","summary_x"]]

df_new = df_merged[['asin','summaryReview','overallScore',"totalReviewers"]]

df_merged = df_merged.sort_values(by='totalReviewers', ascending=False)
df_select = df_merged[df_merged.totalReviewers >= 10]

In [212]:
df_product_reviews = df_select.groupby('asin')['summaryReview'].apply(list)
df_product_reviews = pd.DataFrame(df_product_reviews)

In [199]:
df_mean = pd.merge(df_product_reviews, mean, on='asin', how='inner')
df_mean = df_mean[['asin', 'summaryReview', 'overall']]

In [200]:
regEx = re.compile('[^a-z]+')
def cleanReviews(reviewText):
    value = []
    for i in range(len(reviewText)):
        value.append(reviewText[i].lower())
        value[i] = regEx.sub(' ', value[i]).strip()
    return value

In [201]:
df_mean['summaryClean'] = df_mean['summaryReview'].apply(cleanReviews)
#df_mean = df_mean.drop_duplicates(['overall'], keep='last')

In [202]:
df_mean

,asin,summaryReview,overall,summaryClean
0,0000031887,"[Practically Perfect in every way!, Sassy!, My...",4.608696,"[practically perfect in every way, sassy, my d..."
1,1608299953,"[DO NOT BUY IF YOU EVER WANT TO RESELL IT., Ro...",4.307692,"[do not buy if you ever want to resell it, ros..."
2,1617160377,"[Nice system, but way too expensive., Better t...",4.625000,"[nice system but way too expensive, better tha..."
3,B00004SR8W,"[Great idea, but..., Hidden Neck Wallet with V...",4.666667,"[great idea but, hidden neck wallet with velcr..."
4,B00004SR8Z,"[Not bad, a little flimsy, good buy, Very thin...",4.545455,"[not bad a little flimsy, good buy, very thin ..."
5,B00004SR9P,[great product. Made handling bags so much ea...,4.000000,[great product made handling bags so much easi...
6,B00004U1J2,"[good, Buzz Lightyear Jet Pack, okay buzz ligh...",3.923077,"[good, buzz lightyear jet pack, okay buzz ligh..."
7,B00005TQI7,"[Cute!, Super cute!!!!, My girls really enjoy ...",4.500000,"[cute, super cute, my girls really enjoy it, r..."
8,B00006I551,"[REALLY small, great!, Bic lighter of watches,...",4.230769,"[really small, great, bic lighter of watches, ..."
9,B00006XXGO,"[Great Pair of Shoes, They squeak a bit, Very ...",4.666667,"[great pair of shoes, they squeak a bit, very ..."


In [228]:
reviews = df_mean['summaryClean']
reviews = reviews.reset_index().summaryClean
countVector = CountVectorizer(max_features = 300, stop_words='english')

corpus = []
for i in range(len(reviews)):
    string = ' '.join(reviews[i])
    corpus.append(string)
transformedReviews = countVector.fit_transform(corpus)

dfReviews = pd.DataFrame(transformedReviews.A, columns=countVector.get_feature_names())
dfReviews = dfReviews.astype(int)

In [229]:
X = np.array(dfReviews)
tpercent = 0.85
tsize = int(np.floor(tpercent * len(dfReviews)))
dfReviews_train = X[:tsize]
dfReviews_test = X[tsize:]

lentrain = len(dfReviews_train)
lentest = len(dfReviews_test)

In [230]:
neighbor = NearestNeighbors(n_neighbors=4, algorithm='ball_tree').fit(dfReviews_train)

def calc_neighbors(element):
    a = neighbor.kneighbors([element])
    list_similar = a[1]
    
    first_similar = [item[0] for item in list_similar]
    first_similar = int(str(first_similar).strip('[]'))

    second_similar = [item[1] for item in list_similar]
    second_similar = int(str(second_similar).strip('[]'))
    
    third_similar = [item[2] for item in list_similar]
    third_similar = int(str(third_similar).strip('[]'))
    
    fourth_similar = [item[3] for item in list_similar]
    fourth_similar = int(str(fourth_similar).strip('[]'))
    
#     fifth_similar = [item[4] for item in list_similar]
#     fifth_similar = int(str(fifth_similar).strip('[]'))
    
    return [first_similar, second_similar, third_similar, fourth_similar]

In [244]:
#calc_neighbors(dfReviews_test[0])

In [245]:
df5_train_target = df_mean["overall"][:lentrain]
df5_test_target = df_mean["overall"][lentrain:lentrain+lentest]
df5_train_target = df5_train_target.astype(int)
df5_test_target = df5_test_target.astype(int)

n_neighbors = 4
knnclf = neighbors.KNeighborsClassifier(n_neighbors, weights='distance')
knnclf.fit(dfReviews_train, df5_train_target)
knnpreds_test = knnclf.predict(dfReviews_test)

print(classification_report(df5_test_target, knnpreds_test))

             precision    recall  f1-score   support

          1       0.00      0.00      0.00         1
          2       0.50      0.11      0.18        28
          3       0.45      0.21      0.28       378
          4       0.72      0.91      0.80       897
          5       1.00      0.12      0.22         8

avg / total       0.64      0.68      0.64      1312



/Users/rpss/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [233]:
print(accuracy_score(df5_test_target, knnpreds_test))

0.6829268292682927


In [234]:
print(mean_squared_error(df5_test_target, knnpreds_test))

0.3513719512195122
